In [22]:
from ehrmonize import EHRmonize
import yaml
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

In [25]:
with open('experiments/configs/experiments.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

In [23]:
with open('experiments/configs/get_generic_name.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value

# create dataframe to save all the settings and results
results = pd.DataFrame(columns=[
    'source_db', 'model_id', 'temperature',
    'number_of_shots', 'number_of_attempts', 'agentic',
    'accuracy', 'consistencies', 'time_taken'
])


for db, path, drugn, gt in zip(source_db, source_csv, input_col_name, gt_col_name):
    data = pd.read_csv(path).head(10) # to be based on all the data
    data[gt] = 'heparin' # to be replaced with the actual ground truth
    
    # different engines
    for m in model_id:
        for t in temperature:
            ehrm = EHRmonize(model_id=m, temperature=t)

            # different prompting strategies
            for n_s in number_of_shots:
                for n_a in number_of_attempts:
                    if n_a == 1: # just do once
                        agentic_loop = [False]
                    elif n_a > 1: # go back to agentic defined in the yaml file
                        agentic_loop = agentic

                    for a in agentic_loop:

                        start_time = time.time()

                        ehrm.config_prompting(
                            n_attempts=n_a,
                            n_shots=n_s,
                            agentic=a
                        )
                        print(f'Prompting strategy: {n_a} attempts, {n_s} shots, agentic={a}')

                        ehrm.set_task(
                            task='get_generic_name',
                            possible_shots=possible_shots
                        )

                        if n_a == 1:
                            preds = ehrm.predict(data[drugn])
                            consistencies = np.nan

                        elif n_a > 1:
                            res = ehrm.predict(data[drugn])
                            preds = res['pred']
                            consistencies = res['consistency'].mean()

                        end_time = time.time()

                        results = pd.concat([results, pd.DataFrame({
                            'source_db': [db],
                            'model_id': [m],
                            'temperature': [t],
                            'number_of_shots': [n_s],
                            'number_of_attempts': [n_a],
                            'agentic': [a],
                            'accuracy': [(preds == data[gt]).mean()],
                            'consistencies': [consistencies],
                            'time_taken': [end_time - start_time]
                        })], axis=0)

                        results.to_csv(f"experiments/{db}/{task}.csv", index=False)



Prompting strategy: 1 attempts, 0 shots, agentic=False
Prompting strategy: 3 attempts, 0 shots, agentic=True
Prompting strategy: 3 attempts, 0 shots, agentic=False
Prompting strategy: 5 attempts, 0 shots, agentic=True
Prompting strategy: 5 attempts, 0 shots, agentic=False
Prompting strategy: 1 attempts, 1 shots, agentic=False
Prompting strategy: 3 attempts, 1 shots, agentic=True


KeyboardInterrupt: 

In [24]:
results 

,source_db,model_id,temperature,number_of_shots,number_of_attempts,agentic,accuracy,consistencies,time_taken
0,MIMIC-IV,gpt-3.5-turbo,0,0,1,False,0.1,NaN,3.918149
0,MIMIC-IV,gpt-3.5-turbo,0,0,3,True,0.1,1.0,15.875294
0,MIMIC-IV,gpt-3.5-turbo,0,0,3,False,0.1,1.0,14.315360
0,MIMIC-IV,gpt-3.5-turbo,0,0,5,True,0.1,1.0,28.157131
0,MIMIC-IV,gpt-3.5-turbo,0,0,5,False,0.1,1.0,20.012764
0,MIMIC-IV,gpt-3.5-turbo,0,1,1,False,0.1,NaN,4.252415
